In [1]:
from diffusers import DDPMPipeline, DDIMPipeline, DDIMScheduler
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
import tqdm
import PIL
from matplotlib import pyplot as plt

/home/ubunto/software/miniconda3/envs/dif/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubunto/software/miniconda3/envs/dif/lib/python3.9/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [2]:
prompt = ''

device = 'cuda'
seed = 2024
noisy_sample = torch.randn(
    1, 4, 64, 64
).to(device)
# noisy_sample = torch.randn(
#     1, 4, 512, 512
# ).to('cuda:0')
# noisy_sample = torch.randn(
#     1, 4, 56, 56
# ).to('cuda:0')
# prompt.to(device)

In [3]:
def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)

## 填充腐蚀区域（前提要有members的mask）

In [4]:
def times(x, y):
    return x * y
type(times)

function

In [5]:
from PIL import Image, ImageDraw
import random
import numpy as np

# mask_members_np_demo = np.array(Image.open(mask_members_dir))
# print(mask_members_np_demo.shape)

random.seed(4)

########### 定义颜色转换方法 ##############
import imgviz

def colored_mask(mask, save_path=None):
    lbl_pil = Image.fromarray(mask.astype(np.uint8), mode="P")
    colormap = imgviz.label_colormap()
    # print(colormap)
    lbl_pil.putpalette(colormap.flatten())
    if save_path is not None:
        lbl_pil.save(save_path)

    return lbl_pil

## 载入图片

In [6]:
import os
img_dir = '/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/members/images'
img_dir_list = os.listdir(img_dir)


In [7]:
config_dir = '/home/ubunto/Project/konglx/generate/ControlNet/models/stable-diffusion-v1-5'
control_corrosion_trained_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_corrosion_inpainting_h-256_w-256_2024-07-12_09:10:44_seeds-2024/checkpoint-5700/controlnet'
control_members_trained_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_members_inpainting_h-256_w-256_2024-07-12_14:52:34_seeds-2024/checkpoint-3800/controlnet'
# control_depth_trained_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_depth_depth_h-512_w-512_2024-07-16_11:25:38_seeds-2023/checkpoint-5700/controlnet'
control_depth_trained_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_depth_h-512_w-512_2024-07-16_17:42:27_seeds-2024/checkpoint-2000/controlnet'

# controlnet=[controlnet_corrosion,controlnet_members, controlnet_depth]

import os
# img_dir = '/home/ubunto/Project/konglx/generate/ControlNet-v1-1-nightly/training/corrosion_and_crack/corrosion/JPEGImages'
img_dir_list = os.listdir(img_dir)
save_dir = '/home/ubunto/Project/konglx/generate/diffusers/datasets/for_testing/org_corrosion_members_depth_generated_0'
os.makedirs(save_dir, exist_ok=True)
# len(img_dir_list)
controlnet_conditioning_scale_list=[1.0,1.0,1.0]

control_corrosion_dir = control_corrosion_trained_dir
control_members_dir = control_members_trained_dir
# control_depth_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_depth_depth_h-512_w-512_2024-07-16_11:25:38_seeds-2023/checkpoint-5700/controlnet'
control_depth_dir = control_depth_trained_dir
print('mask_members_dir is not None and mask_corrosion_dir is not None and mask_depth_dir is not None')
controlnet_corrosion = ControlNetModel.from_pretrained(control_corrosion_dir)
controlnet_members = ControlNetModel.from_pretrained(control_members_dir)
controlnet_depth = ControlNetModel.from_pretrained(control_depth_dir)
pipeline = StableDiffusionControlNetPipeline.from_pretrained(config_dir, controlnet=[controlnet_corrosion,controlnet_members, controlnet_depth], controlnet_conditioning_scale=controlnet_conditioning_scale_list).to(device)
generator = torch.Generator(device=device).manual_seed(seed)


for img_name in img_dir_list:
    img_name = img_name.split('.')[0]
    mask_corrosion_dir = f'/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/corrosion/conditioning_images/{img_name}.png'
    mask_members_dir = f'/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/members/conditioning_images/{img_name}.png'
    # mask_members_dir = None
    mask_depth_dir = f'/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/depth/conditioning_images/{img_name}.png'
    if mask_members_dir is not None and mask_corrosion_dir is not None and mask_depth_dir is not None:
        try:
            mask_corrosion_pil = load_image(mask_corrosion_dir)
            mask_members_pil = load_image(mask_members_dir)
            mask_depth_pil = load_image(mask_depth_dir)
            validation_image = [mask_corrosion_pil, mask_members_pil, mask_depth_pil]
            image = pipeline(prompt, validation_image, num_inference_steps=20, generator=generator
                            ).images[0]
            image.save(os.path.join(save_dir, f'{img_name}.jpg'))
        # else:
        #     print(img_name)
        except Exception as e:
            pass
        continue
        # plt.figure(figsize=(20, 5))
        # plt.subplot(151)
        # plt.imshow(mask_corrosion_pil)
        # plt.subplot(152)
        # plt.imshow(mask_members_pil)
        # print(mask_members_pil.mode)
        # plt.subplot(153)
        # plt.imshow(mask_depth_pil)
        # print(mask_depth_pil.mode)
        # plt.subplot(154)
        # plt.imshow(org_img_pil)
        # plt.subplot(155)
        # plt.imshow(image)
        # plt.show()
    # 无members，无corrosion，无depth
    else:
        print('Input nothing')
    # image

Keyword arguments {'controlnet_conditioning_scale': [1.0, 1.0, 1.0]} are not expected by StableDiffusionControlNetPipeline and will be ignored.


mask_members_dir is not None and mask_corrosion_dir is not None and mask_depth_dir is not None


100%|██████████| 20/20 [00:03<00:00,  6.53it/s]
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
100%|██████████| 20/20 [00:03<00:00,  6.44it/s]
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
100%|██████████| 20/20 [00:03<00:00,  6.44it/s]


In [ ]:
mask_corrosion_np = np.array(mask_corrosion_pil)
mask_corrosion_np.min(), mask_corrosion_np.max(), mask_corrosion_np.shape

In [ ]:
# if mask_members_dir is not None and mask_corrosion_dir is None:

#     plt.figure(figsize=(16, 8))
#     plt.subplot(121)
#     plt.imshow(mask_members_pil)
#     plt.subplot(122)
#     plt.imshow(image)
#     plt.show()
#     # Image.show(image)
    
# elif mask_members_dir is  None and mask_corrosion_dir is not None:

#     plt.figure(figsize=(16, 8))
#     plt.subplot(121)
#     plt.imshow(mask_corrosion_pil)
#     plt.subplot(122)
#     plt.imshow(image)
#     plt.show()
# else:

#     plt.figure(figsize=(16, 8))
#     plt.subplot(131)
#     plt.imshow(mask_corrosion_pil)
#     plt.subplot(132)
#     plt.imshow(mask_members_pil)
#     plt.subplot(133)
#     plt.imshow(image)
#     plt.show()

## PNDM

In [ ]:
# PNDMScheduler
generator = torch.Generator(device=device).manual_seed(seed)
image = pipeline(prompt, validation_image, num_inference_steps=20, generator=generator
                ).images[0]
image

## DDPM

In [ ]:
# DDPMScheduler
from diffusers import DDPMScheduler
import os
generator = torch.Generator(device=device).manual_seed(seed)
scheduler_ddpm = DDPMScheduler.from_pretrained(os.path.join(config_dir, 'scheduler'))

# print(pipeline.scheduler)
pipeline.scheduler = scheduler_ddpm
# print(pipeline.scheduler)
image = pipeline(
                   prompt, validation_image, num_inference_steps=20, generator=generator
                ).images[0]
image

In [ ]:
if mask_members_dir is not None and mask_corrosion_dir is None:

    plt.figure(figsize=(16, 8))
    plt.subplot(131)
    plt.imshow(mask_members_pil)
    plt.subplot(132)
    plt.imshow(org_img_pil)
    plt.subplot(133)
    plt.imshow(image)
    plt.show()
    # Image.show(image)
    
elif mask_members_dir is  None and mask_corrosion_dir is not None:

    plt.figure(figsize=(16, 8))
    plt.subplot(131)
    plt.imshow(mask_corrosion_pil)
    plt.subplot(132)
    plt.imshow(org_img_pil)
    plt.subplot(133)
    plt.imshow(image)
    plt.show()
else:

    plt.figure(figsize=(16, 8))
    plt.subplot(141)
    plt.imshow(mask_corrosion_pil)
    plt.subplot(142)
    plt.imshow(mask_members_pil)
    plt.subplot(143)
    plt.imshow(org_img_pil)
    plt.subplot(144)
    plt.imshow(image)
    plt.show()

## DDIM

In [ ]:
# DDIMScheduler
from diffusers import DDIMScheduler
import os
generator = torch.Generator(device=device).manual_seed(seed)
scheduler_ddim = DDIMScheduler.from_pretrained(os.path.join(config_dir, 'scheduler'))

print(pipeline.scheduler)
pipeline.scheduler = scheduler_ddim
print(pipeline.scheduler)
image = pipeline(
                   prompt, validation_image, num_inference_steps=20, generator=generator
                ).images[0]
image

## PNDM

In [ ]:
# PNDMScheduler
from diffusers import PNDMScheduler
import os
generator = torch.Generator(device=device).manual_seed(seed)
scheduler_pndm = PNDMScheduler.from_pretrained(os.path.join(config_dir, 'scheduler'))

print(pipeline.scheduler)
pipeline.scheduler = scheduler_pndm
print(pipeline.scheduler)
image = pipeline(
                   prompt, validation_image, num_inference_steps=20, generator=generator
                ).images[0]
image

# ***##step by step denoise##***

In [ ]:

control_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_crack_only_generate/checkpoint-2000/controlnet'
config_dir = '/home/ubunto/Project/konglx/generate/ControlNet/models/stable-diffusion-v1-5'

## 1.创建各个部分的模型

In [ ]:
from PIL import Image
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler, ControlNetModel
import os

vae = AutoencoderKL.from_pretrained(config_dir, subfolder='vae', use_safetensors=None)
tokenizer = CLIPTokenizer.from_pretrained(config_dir, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(config_dir, subfolder='text_encoder', use_safetensors=None)
unet = UNet2DConditionModel.from_pretrained(config_dir, subfolder='unet', use_safetensors=None)

controlnet = ControlNetModel.from_pretrained(control_dir)
controlnet.config

In [ ]:
vae.config.scaling_factor

In [ ]:
from diffusers import UniPCMultistepScheduler

scheduler_multistep = UniPCMultistepScheduler.from_pretrained(config_dir, subfolder="scheduler")
scheduler_multistep

In [ ]:
torch_device = "cuda"
vae.to(torch_device)
text_encoder.to(torch_device)
unet.to(torch_device)
controlnet.to(torch_device)
unet.config

## 2.Create embeddings

### 2.1 Create text embeddings

In [ ]:
prompt = ["crack"]
device = 'cuda'
seed = 0
height = 512  # default height of Stable Diffusion
width = 512  # default width of Stable Diffusion
num_inference_steps = 25  # Number of denoising steps
guidance_scale = 7.5  # Scale for classifier-free guidance
generator = torch.Generator(device=device).manual_seed(seed)  # Seed generator to create the initial latent noise
batch_size = len(prompt)

In [ ]:
text_input = tokenizer(
    prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt"
)

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

prompt_embeds = text_embeddings
encoder_hidden_states_control = text_embeddings
text_embeddings.shape

### 2.2 Create image embeddings

In [ ]:
from torchvision import transforms

mask_dir = '/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/crack（复件）/SegmentationClass/DeepCrack_11240-6.png'
validation_image = Image.open(mask_dir).convert("RGB")
validation_image.size

conditioning_image_transforms = transforms.Compose(
        [
            transforms.Resize(512, interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(512),
            transforms.ToTensor(),
        ]
    )


conditioning_image_pil = validation_image.resize([512, 512])
conditioning_pixel_values = torch.stack([conditioning_image_transforms(conditioning_image_pil)])
conditioning_pixel_values = conditioning_pixel_values.to(memory_format=torch.contiguous_format).float()
print(conditioning_pixel_values.shape)
controlnet_image = conditioning_pixel_values.to(torch_device)
# print(controlnet_image)
conditioning_image_pil

In [ ]:
'''
You’ll also need to generate the unconditional text embeddings which are the embeddings for the padding token.
These need to have the same shape (batch_size and seq_length) as the conditional text_embeddings:
'''
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer([""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt")
uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]
uncond_embeddings.shape, text_embeddings.shape

In [ ]:
# Let’s concatenate the conditional and unconditional embeddings into a batch to avoid doing two forward passes:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
# text_embeddings
text_embeddings.shape

## 3.Create random noise

In [ ]:
''' 
Next, generate some initial random noise as a starting point for the diffusion process. 
This is the latent representation of the image, and it’ll be gradually denoised. 
At this point, the latent image is smaller than the final image size but that’s okay though 
because the model will transform it into the final 512x512 image dimensions later.
'''

# The height and width are divided by 8 because the vae model has 3 down-sampling layers.
# You can check by running the following:   2 ** (len(vae.config.block_out_channels) - 1) == 8

do_classifier_free_guidance = False
guess_mode = False

latents = torch.randn(
    (batch_size, unet.config.in_channels, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)



latents.shape

In [ ]:
down_block_res_samples, mid_block_res_sample = controlnet(
                    latents,
                    2,
                    encoder_hidden_states=encoder_hidden_states_control,
                    controlnet_cond=controlnet_image,
                    return_dict=False,
                )
len(down_block_res_samples), down_block_res_samples[-1].shape, len(mid_block_res_sample), mid_block_res_sample[0].shape

## 4.Denoise the image

In [ ]:
'''
Start by scaling the input with the initial noise distribution, sigma, the noise scale value, 
which is required for improved schedulers like UniPCMultistepScheduler: 
'''
print(scheduler_multistep.init_noise_sigma)
latents = latents * scheduler_multistep.init_noise_sigma
latents.shape

In [ ]:
''' 
The last step is to create the denoising loop that’ll progressively 
transform the pure noise in latents to an image described by your prompt.
Remember, the denoising loop needs to do three things:

1.Set the scheduler’s timesteps to use during denoising.
2.Iterate over the timesteps.
3.At each timestep, call the UNet model to predict the noise residual and 
pass it to the scheduler to compute the previous noisy sample.

'''

from tqdm.auto import tqdm

scheduler_multistep.set_timesteps(num_inference_steps)

for t in tqdm(scheduler_multistep.timesteps):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)

    latent_model_input = scheduler_multistep.scale_model_input(latent_model_input, timestep=t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler_multistep.step(noise_pred, t, latents).prev_sample

In [ ]:
from PIL import Image
from torchvision import transforms

pil_latents = transforms.ToPILImage()(latents.squeeze(0))
pil_latents


## 5.Decode the image

In [ ]:
# scale and decode the image latents with vae
latents_scaled = 1 / vae.config.scaling_factor * latents
# pil_latents_scaled = transforms.ToPILImage()(latents.squeeze(0))
# pil_latents_scaled = transforms.ToPILImage()(latents_scaled.squeeze(0))
with torch.no_grad():
    # image = vae.decode(latents).sample
    image = vae.decode(latents_scaled).sample
print(image.shape)



In [ ]:
image = (image / 2 + 0.5).clamp(0, 1).squeeze()
image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
image = Image.fromarray(image)
image

In [ ]:
''' 
The last step is to create the denoising loop that’ll progressively 
transform the pure noise in latents to an image described by your prompt.
Remember, the denoising loop needs to do three things:

1.Set the scheduler’s timesteps to use during denoising.
2.Iterate over the timesteps.
3.At each timestep, call the UNet model to predict the noise residual and 
pass it to the scheduler to compute the previous noisy sample.

'''

from tqdm.auto import tqdm

scheduler_multistep.set_timesteps(num_inference_steps)

for t in tqdm(scheduler.timesteps):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    # expand the latents if we are doing classifier free guidance
    latent_model_input_control = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
    # latent_model_input_control = torch.cat([latents_control] * 2)

    latent_model_input_control = scheduler.scale_model_input(latent_model_input_control, timestep=t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred_control = unet(latent_model_input_control, t, encoder_hidden_states=text_embeddings).sample

    # perform guidance
    noise_pred_uncond_control, noise_pred_text_control = noise_pred_control.chunk(2)
    noise_pred_control = noise_pred_uncond_control+ guidance_scale * (noise_pred_text_control - noise_pred_uncond_control)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred_control, t, latents).prev_sample

In [ ]:
a = any
a in 'abc'